In [125]:
import pandas as pd
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [126]:
match=pd.read_csv("data 2/Qatar2022_match_Clean.csv")
equipe = pd.read_csv("data 2/Qatar2022_paticipant_Clean.csv", sep=";")
classement = pd.read_csv("data 2/classement_fifa_2022_Clean.csv")
data_match = pd.read_csv("data 2/lesmatches_internationaux_Clean.csv", sep=";")
dict_table = pickle.load(open('data 2/dict_table','rb'))

In [127]:
classement.head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,1,Germany,GER,57.0,0.0,0,UEFA,1992-12-31
1,96,Syria,SYR,11.0,0.0,0,AFC,1992-12-31
2,97,Burkina Faso,BFA,11.0,0.0,0,CAF,1992-12-31
3,99,Latvia,LVA,10.0,0.0,0,UEFA,1992-12-31
4,100,Burundi,BDI,10.0,0.0,0,CAF,1992-12-31


In [128]:
data_match.head()

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
0,08/08/1993,Bolivia,Uruguay,South America,South America,59,22,0,0,3,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,08/08/1993,Brazil,Mexico,South America,North America,8,14,0,0,1,...,No,Draw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,08/08/1993,Ecuador,Venezuela,South America,South America,35,94,0,0,5,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,08/08/1993,Guinea,Sierra Leone,Africa,Africa,65,86,0,0,1,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,08/08/1993,Paraguay,Argentina,South America,South America,67,5,0,0,1,...,No,Lose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
match.head()

,home,match,away
0,Qatar,Match 1,Ecuador
1,Senegal,Match 2,Netherlands
2,Qatar,Match 18,Senegal
3,Netherlands,Match 19,Ecuador
4,Ecuador,Match 35,Senegal


In [130]:
equipe.head()

,Team,Group
0,Senegal,A
1,Qatar,A
2,Netherlands,A
3,Ecuador,A
4,Iran,B


In [133]:
data_match_df = data_match.replace({"IR Iran": "Iran", "Korea Republic" : "South Korea"})
classement_df = classement.replace({"IR Iran": "Iran", "Korea Republic" : "South Korea"})

In [134]:
data_match_df['rank_difference'] = data_match_df['home_team_fifa_rank'] - data_match_df['away_team_fifa_rank']
data_match_df['average_rank'] = (data_match_df['home_team_fifa_rank'] + data_match_df['away_team_fifa_rank'])/2
data_match_df['point_difference'] = data_match_df['home_team_total_fifa_points'] - data_match_df['away_team_total_fifa_points']
data_match_df['is_stake'] = data_match_df['tournament'] != 'Friendly'
data_match_df['is_worldcup'] = 'FIFA World Cup' in data_match_df['tournament']
data_match_df['score_difference'] = data_match_df['home_team_score'] - data_match_df['away_team_score']
data_match_df['is_won'] = data_match_df['score_difference'] > 0 # Take draw as lost


In [135]:
A= data_match_df.loc[:,['average_rank', 'rank_difference', 'point_difference', 'is_stake', 'is_worldcup']]
B=data_match_df['is_won']
A_train, A_test, B_train, B_test = train_test_split(A, B, test_size=0.2, random_state=42)

In [136]:
A

,average_rank,rank_difference,point_difference,is_stake,is_worldcup
0,40.5,37,0,True,False
1,11.0,-6,0,False,False
2,64.5,-59,0,True,False
3,75.5,-21,0,False,False
4,36.0,62,0,True,False
...,...,...,...,...,...
23916,166.5,27,-108,True,False
23917,163.5,57,-210,True,False
23918,44.0,-32,139,True,False
23919,29.0,-12,54,True,False


In [137]:
B

0         True
1        False
2         True
3         True
4        False
         ...  
23916     True
23917    False
23918    False
23919    False
23920     True
Name: is_won, Length: 23921, dtype: bool

Support Vector Machines
Random Forest

In [138]:
A_train

,average_rank,rank_difference,point_difference,is_stake,is_worldcup
10294,24.0,14,0,False,False
273,40.0,-36,0,True,False
22264,37.5,-35,148,True,False
6916,64.5,43,0,True,False
7107,49.5,-69,0,True,False
...,...,...,...,...,...
21575,162.5,-59,184,True,False
5390,110.5,-19,0,False,False
860,143.5,1,0,True,False
15795,164.5,-11,42,False,False


In [139]:
A_test

,average_rank,rank_difference,point_difference,is_stake,is_worldcup
2476,17.5,33,0,True,False
8225,48.0,94,0,False,False
20053,87.0,-54,278,True,False
21512,147.0,-6,23,True,False
4104,147.5,-1,0,False,False
...,...,...,...,...,...
1258,53.0,-10,0,True,False
20856,65.5,9,-36,False,False
23818,139.5,59,-198,True,False
12586,43.0,-68,0,True,False


In [140]:
B_train

10294     True
273       True
22264     True
6916     False
7107      True
         ...  
21575     True
5390     False
860      False
15795     True
23654    False
Name: is_won, Length: 19136, dtype: bool

In [141]:
B_test

2476     False
8225     False
20053    False
21512    False
4104      True
         ...  
1258      True
20856    False
23818    False
12586     True
19700    False
Name: is_won, Length: 4785, dtype: bool

In [142]:
# Support Vector Machines

svc = SVC()
svc.fit(A_train, B_train) #pour les variables d'entrainemenet
svm_pred = svc.predict(A_test)
accurrancy= round(svc.score(A_test, B_test) , 2)
accurrancy

0.68

In [143]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=50)
random_forest.fit(A_train, B_train)  #pour les variables d'entrainemenet
Y_pred = random_forest.predict(A_test)
random_forest.score(A_train, B_train)
accurrancy = round(random_forest.score(A_test, B_test), 2)
accurrancy

0.64

In [144]:

classement_df = classement_df.loc[(classement_df['rank_date'] == classement_df['rank_date'].max()) & (classement_df['country_full'].isin(equipe['Team'].unique()))]

classement_coup_du_monde = classement_df.set_index(['country_full'])

In [147]:
country_win_prob = {}
for country in equipe['Team'].unique():
    country_win_prob[country] = list()

country_win_prob

{'Senegal': [],
 'Qatar': [],
 'Netherlands': [],
 'Ecuador': [],
 'Iran': [],
 'England': [],
 'USA': [],
 'Wales': [],
 'Argentina': [],
 'Saudi Arabia': [],
 'Mexico': [],
 'Poland': [],
 'Denmark': [],
 'Tunisia': [],
 'France': [],
 'Australia': [],
 'Germany': [],
 'Japan': [],
 'Spain': [],
 'Costa Rica': [],
 'Morocco': [],
 'Croatia': [],
 'Belgium': [],
 'Canada': [],
 'Switzerland': [],
 'Cameroon': [],
 'Brazil': [],
 'Serbia': [],
 'Uruguay': [],
 'South Korea': [],
 'Portugal': [],
 'Ghana': []}

# Group Stage Match Prediction

In [148]:
def predict_points(home, away):

    # creer et initailiser les lignes et les column porte `average_rank', 'rank_difference', 'point_difference', 'is_stake', 'is_worldcup
        ligne = pd.DataFrame(np.array([[np.nan, np.nan, np.nan, True, True]]), columns=A_train.columns)

        home_rank = classement_coup_du_monde.loc[home, 'rank']
        home_points = classement_coup_du_monde.loc[home, 'total_points']
        away_rank = classement_coup_du_monde.loc[away, 'rank']
        away_points = classement_coup_du_monde.loc[away, 'total_points']

        ligne['average_rank'] = (home_rank + away_rank) / 2
        ligne['rank_difference'] = home_rank - away_rank
        ligne['point_difference'] = home_points - away_points

        modele = svc.predict(ligne)

        awaypoints=0
        homepoints=0

        if modele == False:
            awaypoints = 3

        elif modele== True:
            homepoints= 3

        return homepoints,awaypoints

diviser les phases de la competition

In [149]:
phase_de_groupes = match[:48].copy()
huitième_de_finale = match[48:56].copy()
quarts_de_finale = match[56:60].copy()
demi_finale= match[60:62].copy()
final = match[62:].copy()

In [150]:
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    match_group = phase_de_groupes[phase_de_groupes['home'].isin(teams_in_group)]
    for index, row in match_group.iterrows():
        home, away = row['home'], row['away']
        if home=='United States'or away=='United States':
            points_home=0
            points_away=0
        else:
            points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]



In [153]:
dict_table


{'Group A':           Team  Pts
 0  Netherlands    9
 1      Senegal    3
 2    Qatar (H)    0
 3      Ecuador    0,
 'Group B':             Team  Pts
 0        England    6
 1           Iran    3
 2  United States    0
 3          Wales    0,
 'Group C':            Team  Pts
 0     Argentina    9
 1        Mexico    6
 2        Poland    3
 3  Saudi Arabia    0,
 'Group D':         Team  Pts
 0     France    9
 1    Denmark    6
 2  Australia    3
 3    Tunisia    0,
 'Group E':          Team  Pts
 0       Spain    9
 1     Germany    6
 2       Japan    3
 3  Costa Rica    0,
 'Group F':       Team  Pts
 0  Belgium    9
 1  Croatia    6
 2  Morocco    3
 3   Canada    0,
 'Group G':           Team  Pts
 0       Brazil    9
 1  Switzerland    6
 2       Serbia    3
 3     Cameroon    0,
 'Group H':           Team  Pts
 0      Uruguay    9
 1     Portugal    6
 2  South Korea    3
 3        Ghana    0}

In [154]:
group_winner={}
for group in dict_table:
    group_winner[group] = dict_table[group].loc[0, 'Team'],dict_table[group].loc[1, 'Team']

group_winner

{'Group A': ('Netherlands', 'Senegal'),
 'Group B': ('England', 'Iran'),
 'Group C': ('Argentina', 'Mexico'),
 'Group D': ('France', 'Denmark'),
 'Group E': ('Spain', 'Germany'),
 'Group F': ('Belgium', 'Croatia'),
 'Group G': ('Brazil', 'Switzerland'),
 'Group H': ('Uruguay', 'Portugal')}

In [155]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    huitième_de_finale.replace({f'Winners {group}':group_winner,
                                 f'Runners-up {group}':runners_up}, inplace=True)

huitième_de_finale['winner'] = '?'
huitième_de_finale

,home,match,away,winner
48,Netherlands,Match 49,Iran,?
49,Argentina,Match 50,Denmark,?
50,France,Match 52,Mexico,?
51,England,Match 51,Senegal,?
52,Spain,Match 53,Croatia,?
53,Brazil,Match 54,Portugal,?
54,Belgium,Match 55,Germany,?
55,Uruguay,Match 56,Switzerland,?


In [156]:
def predict_victory(home, away):

    # creer et initailiser les lignes et les column porte `average_rank', 'rank_difference', 'point_difference', 'is_stake', 'is_worldcup
    ligne = pd.DataFrame(np.array([[np.nan, np.nan, np.nan, True, True]]), columns=A_train.columns)

    home_rank = classement_coup_du_monde.loc[home, 'rank']
    home_points = classement_coup_du_monde.loc[home, 'total_points']
    away_rank = classement_coup_du_monde.loc[away, 'rank']
    away_points = classement_coup_du_monde.loc[away, 'total_points']

    ligne['average_rank'] = (home_rank + away_rank) / 2
    ligne['rank_difference'] = home_rank - away_rank
    ligne['point_difference'] = home_points - away_points

    modele = random_forest.predict(ligne)

    if modele == False:
        victory = 'away'

    elif modele== True:
        victory = 'home'

    return  victory

In [157]:
def get_winner(match):
    for index, row in match.iterrows():
        home, away = row['home'], row['away']
        winn = predict_victory(home, away)
        if winn=='home':
            winner = home
        else:
            winner = away
        match.loc[index, 'winner'] = winner
    return match

In [158]:
get_winner(huitième_de_finale)

,home,match,away,winner
48,Netherlands,Match 49,Iran,Netherlands
49,Argentina,Match 50,Denmark,Argentina
50,France,Match 52,Mexico,Mexico
51,England,Match 51,Senegal,England
52,Spain,Match 53,Croatia,Spain
53,Brazil,Match 54,Portugal,Portugal
54,Belgium,Match 55,Germany,Belgium
55,Uruguay,Match 56,Switzerland,Switzerland


In [159]:
def update_table(round_1,round_2):
    for index, row in round_1.iterrows():
        winner = round_1.loc[index, 'winner']
        match = round_1.loc[index, 'match']
        round_2.replace({f'Winners {match}':winner}, inplace=True)
    round_2['winner'] = '?'
    return round_2

In [160]:
update_table(huitième_de_finale, quarts_de_finale)

,home,match,away,winner
56,Spain,Match 58,Portugal,?
57,Netherlands,Match 57,Argentina,?
58,Belgium,Match 60,Switzerland,?
59,England,Match 59,Mexico,?


In [161]:
get_winner(quarts_de_finale)

,home,match,away,winner
56,Spain,Match 58,Portugal,Portugal
57,Netherlands,Match 57,Argentina,Argentina
58,Belgium,Match 60,Switzerland,Belgium
59,England,Match 59,Mexico,England


In [162]:
update_table(quarts_de_finale, demi_finale)

,home,match,away,winner
60,Argentina,Match 61,Portugal,?
61,England,Match 62,Belgium,?


In [163]:
get_winner(demi_finale)

,home,match,away,winner
60,Argentina,Match 61,Portugal,Portugal
61,England,Match 62,Belgium,Belgium


In [164]:
update_table(demi_finale,final)

,home,match,away,winner
62,Portugal,Match 64,Belgium,?


In [165]:
get_winner(final)

,home,match,away,winner
62,Portugal,Match 64,Belgium,Belgium


In [166]:
# Merci pour votre attention